In [ ]:
# Open the file in read mode
with open('all.txt', 'r') as file:
    # Read the contents of the file
    text = file.read()

In [ ]:
import os
from crewai import Agent, Task, Process, Crew

import ollama 
# from langchain.agents import initialize_agent
from langchain.llms.ollama import Ollama

from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI


import os 
os.environ["OPENAI_API_KEY"] = "sk-xyz"

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
embeddings=Ollama(model="nomic-embed-text:latest")

from interpreter import interpreter
interpreter.llm.model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
interpreter.llm.embedding="nomic-embed-text:latest"


# llm = Ollama(model="qwen2.5:0.5b ", verbose = True)
# embeddings=Ollama(model="nomic-embed-text:latest")

# from interpreter import interpreter
# interpreter.llm.max_tokens = 8196
# interpreter.llm.context_window = 128000
# interpreter.auto_run = True
# interpreter.llm.model = "qwen2.5:0.5b "
# interpreter.llm.embedding="nomic-embed-text:latest"

In [ ]:
def summary_agent(text):
    code_analyst = Agent(
        role='Transcript Summarizer',
        goal=f"To efficiently and accurately summarize transcripts by analysing the content and make it subject specific",
        backstory="This agent is a seasoned summarizer.This agent should be capable of reading and understanding the given text, extracting key points, and summarizing them in a coherent, concise manner. Their expertise lies in dissecting the complex methdology and technical information presented and generate a succient summary.",
        verbose=True,
        allow_delegation=False,
        llm=llm
    )

    analyse_code = Task(
        description="""This agent should be capable of reading and understanding the given text, extracting key points, and summarizing them in a coherent, concise manner for anyone to understand
        text:{text}
        DONOT USE ANY ACRONYMNS, ABBREVIATIONS, EXPLAIN EVERYTHING IN THE SIMPLEST LANGUGAE TO A PROGRAM MANAGER
        IF THERE ARE ANY EQUATIONS PROVIDE THEM IN A PROPER FORMAT AND EXPLAIN THEM
        """,
        expected_output=f"""
        Detailed summary of the technical and scientific research text provided targeted for a buisness analyst to understand in the most simplest words.IF THERE ARE ANY EQUATIONS PROVIDE THEM IN A PROPER FORMAT AND EXPLAIN THEM
        """,
        agent=code_analyst
    )

    crew = Crew(
        agents=[code_analyst],
        tasks=[analyse_code],
        manager_llm=llm,
        memory=True,
        embedder={
            "provider": "ollama",
            "config": {
                "model": 'nomic-embed-text'
            }
        }
    )

    inputs = {"text": text}
    return crew.kickoff(inputs=inputs)


In [ ]:
summary=summary_agent(text)